In [1]:
import os
import random
import shutil
import json
from PIL import Image
import cv2
from ultralytics import YOLO
import pandas as pd

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 


Cambio de nombre de las imágenes

In [4]:
# Número actual de imágenes
imagenes_actuales = 125

# Ruta de la carpeta original que contiene las imágenes
ruta_original = "imagenes"

# Nombre de la nueva carpeta donde se guardarán las imágenes renombradas
nueva_carpeta = "Dataset/"

# Asegurarse de que la nueva carpeta existe o se crea
if not os.path.exists(nueva_carpeta):
    os.makedirs(nueva_carpeta)

# Recorrer las imágenes en la carpeta original
for i, nombre_archivo in enumerate(os.listdir(ruta_original)):
    # Verificar si el archivo es una imagen (puedes añadir más extensiones si lo necesitas)
    if nombre_archivo.endswith(('.jpg')):
        # Definir el nuevo nombre para la imagen (ejemplo: imagen_1.jpg)
        nuevo_nombre = f"imagen_{i+1+imagenes_actuales}.jpg"

        # Ruta completa del archivo original
        ruta_original_completa = os.path.join(ruta_original, nombre_archivo)

        # Ruta completa del nuevo archivo
        ruta_nueva_completa = os.path.join(nueva_carpeta, nuevo_nombre)

        # Copiar y renombrar la imagen a la nueva carpeta
        shutil.copy(ruta_original_completa, ruta_nueva_completa)

print("Proceso completado.")


Proceso completado.


Limpieza de los directorios train, val y test para añadir imagenes nuevamente

In [10]:
# Función para limpiar los directorios de imágenes y etiquetas
def limpiar_directorios(carpeta_destino):
    # Subcarpetas para los conjuntos de datos
    subcarpetas = ['train', 'test', 'val']
    
    # Tipos de archivo que queremos eliminar
    extensiones = ['.jpg', '.png', '.txt']  # Añade aquí otras extensiones de imagen si es necesario
    
    for subcarpeta in subcarpetas:
        carpeta_imagenes = os.path.join(carpeta_destino, subcarpeta, "images")
        carpeta_labels = os.path.join(carpeta_destino, subcarpeta, "labels")
        
        # Eliminar archivos en la carpeta de imágenes
        if os.path.exists(carpeta_imagenes):
            for archivo in os.listdir(carpeta_imagenes):
                if any(archivo.endswith(ext) for ext in extensiones):
                    os.remove(os.path.join(carpeta_imagenes, archivo))
        
        # Eliminar archivos en la carpeta de etiquetas
        if os.path.exists(carpeta_labels):
            for archivo in os.listdir(carpeta_labels):
                if archivo.endswith('.txt'):
                    os.remove(os.path.join(carpeta_labels, archivo))
    
    print("Los directorios de train, test y val han sido limpiados.")

# Ejemplo de uso
carpeta_destino = "VC_P4-NUM_PLATES"  # Ruta a la carpeta principal
limpiar_directorios(carpeta_destino)

Los directorios de train, test y val han sido limpiados.


Distribución de las imágenes entre train (70%), test (20%) y val (10%).

Tarea: Detector de Personas, coches y matrículas. 

In [1]:
from collections import defaultdict
import numpy as np
import cv2
import torch
import math
from ultralytics import YOLO
import easyocr
import csv
import os

# Initialize YOLO and EasyOCR models
model_detection = YOLO('yolo11n.pt')  # Model for detecting people and cars
model_license_plate = YOLO('runs/detect/train/weights/best.pt')  # License plate detection model
reader = easyocr.Reader(['en'])  # EasyOCR for license plate recognition

# Class labels
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus"]

# Initialize counters and sets to track unique detections
counts = {'person': 0, 'car': 0, 'plate': 0, 'bus':0}
seen_ids = {'person': set(), 'car': set(), 'plate': set(), 'bus':set()}

# Output CSV file for detection details
csv_file = 'resultados_detalles.csv'
if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Tipo_objeto', 'Confianza', 'ID_Tracking', 'x1', 'y1', 'x2', 'y2',
                         'Matrícula', 'Confianza_Matrícula', 'mx1', 'my1', 'mx2', 'my2', 'Texto_Matrícula'])

# Capture video
vid = cv2.VideoCapture("C0142.MP4")
frame_width, frame_height = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))

# Initialize video writer for output video
output_video = 'output_video_with_results.mp4'
output_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_number = 0

while True:
    ret, img = vid.read()
    if not ret:
        break
    
    frame_number += 1

    # Detect people and cars
    results = model_detection.track(img, persist=True, classes=[0, 2, 5])  # Detect classes 0 (person), 2 (car), and 5 (bus)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            cls = int(box.cls[0])  # Class label
            confidence = float(box.conf[0])  # Confidence score
            object_type = classNames[cls] if cls < len(classNames) else 'unknown'

            # Tracking ID
            track_id = str(int(box.id[0].tolist())) if box.id is not None else ''

            # Check if the object has already been counted
            if track_id and track_id not in seen_ids[object_type]:
                # Count only new appearances
                counts[object_type] += 1
                seen_ids[object_type].add(track_id)  # Add the ID to the set of seen IDs

            if object_type in ["car", "bus"]:
                # Extract the car region for license plate detection
                car_img = img[y1:y2, x1:x2]
                plate_results = model_license_plate(car_img)

                # Draw plate detections on the image
                for plate in plate_results:
                    
                    plate_boxes = plate.boxes
                    for pbox in plate_boxes:
                        px1, py1, px2, py2 = map(int, pbox.xyxy[0])

                        # Crop and preprocess license plate image for OCR
                        plate_img = car_img[py1:py2, px1:px2]
                        if plate_img.size == 0:
                            continue
                        
                        if track_id and track_id not in seen_ids['plate']:
                        # Count only new appearances
                            counts['plate'] += 1
                            seen_ids['plate'].add(track_id)  # Add the ID to the set of seen IDs

                        # Resize and apply CLAHE and thresholding
                        scale_factor = 1.5
                        resized_plate = cv2.resize(plate_img, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
                        gray_plate = cv2.cvtColor(resized_plate, cv2.COLOR_BGR2GRAY)
                        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
                        contrast_plate = clahe.apply(gray_plate)
                        binary_plate_adaptive = cv2.adaptiveThreshold(contrast_plate, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

                        # Run OCR on the processed plate image
                        try:
                            result = reader.readtext(
                                binary_plate_adaptive,
                                decoder='beamsearch',
                                detail=1,
                                allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
                            )

                            if result:
                                for (bbox, text, prob) in result:
                                    # Log results in CSV and draw on frame
                                    with open(csv_file, mode='a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow([frame_number, object_type, f'{confidence:.2f}', track_id,
                                                         x1, y1, x2, y2, text, f'{prob:.2f}', 
                                                         px1, py1, px2, py2, text])

                                    # Draw detected plate text on image
                                    cv2.putText(img, text, (x1 + px1, y1 + py1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                                    cv2.rectangle(img, (x1 + px1, y1 + py1), (x1 + px2, y1 + py2), (0, 255, 0), 2)
                        except Exception as e:
                            print("Error during OCR:", e)

            # Draw bounding boxes for all objects
            color = (0, 0, 255) if object_type == "person" else (255, 0, 0)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f'{object_type} {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Update frame count overlay
    cv2.putText(img, f'Frame: {frame_number}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(img, f'People: {counts["person"]}, Cars: {counts["car"]}, Plates: {counts["plate"]}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Write annotated frame to output video
    output_writer.write(img)
    
    # Display the video
    cv2.imshow('Video with Detections', img)
    if cv2.waitKey(20) & 0xFF == 27:
        break

# Cleanup
vid.release()
output_writer.release()
cv2.destroyAllWindows()

print(f"Processed video saved as {output_video}")


WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 

0: 384x640 1 car, 1 bus, 59.5ms
Speed: 7.5ms preprocess, 59.5ms inference, 214.5ms postprocess per image at shape (1, 3, 384, 640)

0: 544x640 (no detections), 58.5ms
Speed: 4.0ms preprocess, 58.5ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)

0: 512x640 (no detections), 48.0ms
Speed: 3.5ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 1 car, 1 bus, 11.0ms
Speed: 2.5ms preprocess, 11.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 544x640 1 matricula, 11.5ms
Speed: 2.5ms preprocess, 11.5ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 640)

0: 512x640 (no detections), 13.0ms
Speed: 3.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 1 car, 1 bus, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0